### Analysis of Meridian Results - log linear model


In [2]:
model_name="loglinear"
from utils import clean_numeric_dataframe

In [3]:
import sys, os
IN_COLAB = ('google.colab' in sys.modules) or ('COLAB_RELEASE_TAG' in os.environ)

In [4]:
# Install meridian: from PyPI @ latest release (robust in Colab and local Jupyter)
import sys, subprocess
pkg = "google-meridian[colab,and-cuda]" if IN_COLAB else "google-meridian"
print(f"Installing: {pkg}")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])
except Exception as e:
    print(f"pip install failed for {pkg}: {e}")

Installing: google-meridian


In [5]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import formatter
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.data import data_frame_input_data_builder as data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

if IN_COLAB:
    from google.colab import drive


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)





Your runtime has 16.9 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [44]:
## load from local
file_path = f'Results\\saved_mmm_{model_name}.pkl'
mmm = model.load_mmm(file_path)

In [ ]:
mmm_summarizer_ll = summarizer.Summarizer(mmm)
analyzer_ll = analyzer.Analyzer(mmm)
mediaEffects_ll = visualizer.MediaEffects(mmm)
model_diagnostics_ll = visualizer.ModelDiagnostics(mmm)
model_fit_ll = visualizer.ModelFit(mmm)
media_summary_ll = visualizer.MediaSummary(mmm)


In [ ]:
model_diagnostics_ll.plot_prior_and_posterior_distribution()

In [45]:
import altair as alt
from meridian import constants as c

# def plot_prior_and_posterior_distribution(
# mmm
parameter: str = 'roi_m'
num_geos: int = 3
selected_times: list[str] | None = None
# ) -> alt.Chart | alt.FacetChart:
"""Plots prior and posterior distributions for a model parameter.

Args:
  parameter: Model parameter name to plot. By default, the ROI parameter is
    shown if a name is not specified.
  num_geos: Number of largest geos by population to show in the plots for
    the geo-level parameters. By default, only the top three largest geos
    are shown.
  selected_times: List of specific time periods to plot for time-level
    parameters. These times must match the time periods from the data. By
    default, the first three time periods are plotted.

Returns:
  An Altair plot showing the parameter distributions.

Raises:
  NotFittedModelError: The model hasn't been fitted.
  ValueError: A `parameter` is not a Meridian model parameter.
"""
if not (
    hasattr(mmm._meridian.inference_data, c.PRIOR)
    and hasattr(mmm._meridian.inference_data, c.POSTERIOR)
):
  raise model.NotFittedModelError(
      'Plotting prior and posterior distributions requires fitting the'
      ' model.'
  )

# # Check if the selected parameter is part of Meridian's model parameters.
# if (
#     parameter
#     not in mmm._meridian.inference_data.posterior.data_vars.keys()
# ):
#   raise ValueError(
#       f"The selected param '{parameter}' does not exist in Meridian's model"
#       ' parameters.'
#   )

# if selected_times:
#   param_data = mmm._meridian.inference_data.posterior[parameter]
#   if not (hasattr(param_data, c.TIME)):
#     raise ValueError(
#         '`selected_times` can only be used if the parameter has a time'
#         f" dimension. The selected param '{parameter}' does not have a time"
#         ' dimension.'
#     )
#   if any(time not in param_data.time for time in selected_times):
#     raise ValueError(
#         'The selected times must match the time dimensions in the Meridian'
#         ' model.'
#     )

# prior_dat = mmm._meridian.inference_data.prior[parameter]
# posterior_dat = mmm._meridian.inference_data.posterior[parameter]
# prior_df = (
#     prior_dat.to_dataframe().reset_index().drop(columns=[c.CHAIN, c.DRAW])
# )
# posterior_df = (
#     posterior_dat.to_dataframe()
#     .reset_index()
#     .drop(columns=[c.CHAIN, c.DRAW])
# )

# # Tag the data before combining.
# prior_df[c.DISTRIBUTION] = c.PRIOR
# posterior_df[c.DISTRIBUTION] = c.POSTERIOR
# prior_posterior_df = pd.concat([prior_df, posterior_df])

# if c.GEO in prior_posterior_df.columns:
#   top_geos = mmm._meridian.input_data.get_n_top_largest_geos(num_geos)
#   prior_posterior_df = prior_posterior_df[
#       prior_posterior_df[c.GEO].isin(top_geos)
#   ]

# if c.TIME in prior_posterior_df.columns:
#   default_num_times = 3
#   times = (
#       selected_times
#       if selected_times
#       else prior_dat[c.TIME][:default_num_times].values
#   )
#   prior_posterior_df = prior_posterior_df[
#       prior_posterior_df[c.TIME].isin(times)
#   ]

# groupby = posterior_df.columns.tolist()
# groupby.remove(parameter)
# plot = (
#     alt.Chart(prior_posterior_df, width=c.VEGALITE_FACET_DEFAULT_WIDTH)
#     .transform_density(
#         parameter, groupby=groupby, as_=[parameter, 'density']
#     )
#     .mark_area(opacity=0.7)
#     .encode(
#         x=f'{parameter}:Q',
#         y=alt.Y(shorthand='density:Q', stack=False),
#         color=f'{c.DISTRIBUTION}:N',
#     )
# )

AttributeError: 'Meridian' object has no attribute '_meridian'

In [9]:
model_fit_ll.plot_model_fit(
                         include_baseline=False,
                         include_ci=False)

alt.Chart(...)

In [10]:
df_media_results = media_summary_ll.summary_table()

c:\Users\User\repos\MMM\.venv\Lib\site-packages\meridian\analysis\visualizer.py:1629: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  .aggregate(lambda g: f'{g[0]} ({g[1]}, {g[2]})')


In [11]:
model_diagnostics_ll.predictive_accuracy_table()

c:\Users\User\repos\MMM\.venv\Lib\site-packages\meridian\analysis\analyzer.py:590: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


,metric,geo_granularity,value
0,R_Squared,national,0.938264
1,MAPE,national,0.077042
2,wMAPE,national,0.071232


In [26]:
source_model = ""
base_dir = 'Results'
df_rois = pd.read_csv(os.path.join(base_dir, f'rois{source_model}.csv'))
df_decomp_vol = pd.read_csv(os.path.join(base_dir, f'decomp{source_model}.csv'))
df_var_spec = pd.read_csv(os.path.join(base_dir, f'var_spec{source_model}.csv'))
df_rois = df_rois.rename(columns={'variable': 'channel','spend_sum': 'spend'})

# print('df_rois:', df_rois.shape)
# print('df_decomp_vol:', df_decomp_vol.shape)
# print('df_var_spec:', df_var_spec.shape)
# display(df_rois.head())
# display(df_decomp_vol.head())
# display(df_var_spec.head())

In [20]:
df_media_results

,channel,distribution,impressions,% impressions,spend,% spend,cpm,incremental outcome,% contribution,roi,effectiveness,mroi,cpik
0,m_wow_tv,prior,"2,699,491",15.5%,"$2,699,491",15.5%,"$1,000","$4,557,542 ($655,414, $11,367,260)","0.3% (0.0%, 0.8%)","1.7 (0.2, 4.2)","1.69 (0.24, 4.21)","0.9 (0.1, 2.5)","$7.4 ($2.2, $37.4)"
1,m_wow_tv,posterior,"2,699,491",15.5%,"$2,699,491",15.5%,"$1,000","$1,063,363 ($333,165, $2,238,915)","0.1% (0.0%, 0.2%)","0.4 (0.1, 0.8)","0.39 (0.12, 0.83)","0.2 (0.1, 0.5)","$26.3 ($10.9, $73.4)"
2,m_wow_olv,prior,"947,317",5.4%,"$947,317",5.4%,"$1,000","$1,808,167 ($257,901, $5,415,584)","0.1% (0.0%, 0.4%)","1.9 (0.3, 5.7)","1.91 (0.27, 5.72)","0.8 (0.1, 2.4)","$7.0 ($1.6, $32.9)"
3,m_wow_olv,posterior,"947,317",5.4%,"$947,317",5.4%,"$1,000","$797,647 ($185,931, $1,920,254)","0.1% (0.0%, 0.1%)","0.8 (0.2, 2.0)","0.84 (0.20, 2.03)","0.4 (0.1, 0.9)","$13.4 ($4.4, $45.7)"
4,m_wow_social,prior,"121,919",0.7%,"$121,919",0.7%,"$1,000","$223,848 ($33,008, $629,518)","0.0% (0.0%, 0.0%)","1.8 (0.3, 5.2)","1.84 (0.27, 5.16)","0.5 (0.1, 1.5)","$7.0 ($1.7, $33.0)"
5,m_wow_social,posterior,"121,919",0.7%,"$121,919",0.7%,"$1,000","$186,071 ($31,526, $553,285)","0.0% (0.0%, 0.0%)","1.5 (0.3, 4.5)","1.53 (0.26, 4.54)","0.5 (0.1, 1.4)","$8.5 ($2.0, $34.7)"
6,m_amaze_tot,prior,"7,929,905",45.6%,"$7,929,905",45.6%,"$1,000","$14,406,834 ($2,103,627, $43,349,320)","1.0% (0.1%, 3.0%)","1.8 (0.3, 5.5)","1.82 (0.27, 5.47)","0.9 (0.1, 2.6)","$7.5 ($1.8, $36.4)"
7,m_amaze_tot,posterior,"7,929,905",45.6%,"$7,929,905",45.6%,"$1,000","$5,448,716 ($1,543,110, $11,468,402)","0.4% (0.1%, 0.8%)","0.7 (0.2, 1.4)","0.69 (0.19, 1.45)","0.3 (0.1, 0.7)","$16.0 ($6.7, $49.5)"
8,m_celeb_tv,prior,"2,828,021",16.2%,"$2,828,021",16.2%,"$1,000","$5,580,696 ($751,725, $16,426,540)","0.4% (0.1%, 1.1%)","2.0 (0.3, 5.8)","1.97 (0.27, 5.81)","0.9 (0.1, 2.7)","$7.8 ($1.7, $36.8)"
9,m_celeb_tv,posterior,"2,828,021",16.2%,"$2,828,021",16.2%,"$1,000","$50,508,536 ($31,253,340, $67,180,704)","3.7% (2.3%, 5.0%)","17.9 (11.1, 23.8)","17.86 (11.05, 23.76)","9.9 (4.6, 14.5)","$0.5 ($0.4, $0.9)"


In [21]:
posterior_mask = df_media_results['distribution'].str.lower().str.contains('posterior')
df_post = df_media_results[posterior_mask].copy()
df_post = df_post[['channel','spend','incremental outcome','roi']]


In [22]:
# Clean 'incremental outcome' and 'roi' columns: extract value before bracket, remove $/commas, convert to number
import re

def clean_value(val):
    if pd.isnull(val):
        return None
    # Take value before first bracket
    s = str(val).split('(')[0].strip()
    # Remove $ and commas
    s = re.sub(r'[$,]', '', s)
    try:
        return float(s)
    except Exception:
        return None

cols_to_clean = ['incremental outcome', 'roi']
if all(col in df_post.columns for col in cols_to_clean):
    df_post_clean = df_post.copy()
    for col in cols_to_clean:
        df_post_clean[col] = df_post_clean[col].apply(clean_value)
    display(df_post_clean[['channel', 'spend', 'incremental outcome', 'roi']])
else:
    print("Some required columns missing in df_post. Available columns:", df_post.columns.tolist())

,channel,spend,incremental outcome,roi
1,m_wow_tv,"$2,699,491",1063363.0,0.4
3,m_wow_olv,"$947,317",797647.0,0.8
5,m_wow_social,"$121,919",186071.0,1.5
7,m_amaze_tot,"$7,929,905",5448716.0,0.7
9,m_celeb_tv,"$2,828,021",50508536.0,17.9
11,m_celeb_outdoor,"$1,290,619",13401968.0,10.4
13,m_celeb_display,"$1,586,718",13889397.0,8.8
15,All Channels,"$17,403,992",85295560.0,4.9


In [23]:
df_post = df_post_clean.rename(columns={'spend': 'spend (mer)','incremental outcome':'value (mer)', 'roi':'roi (mer)'})
# df_post[df_post['channel'] == 'All Channels'].loc['channel']="Total"
idx = df_post.index[df_post['channel'] == 'All Channels']
df_post.loc[idx, 'channel'] = 'Total'

In [30]:
merged = df_rois.merge(df_post, on='channel', how='left', suffixes=('', '_rois'))
clean_numeric_dataframe(merged, exclude=['channel', 'variable'], in_place=True)

merged['roi']=merged['roi']
merged['roi (est)']=merged['roi (est)']

# Percent change calculations
merged['%_change_spend'] = 100 * (merged['spend (mer)'] - merged['spend']) / merged['spend']
merged['%_change_value'] = 100 * (merged['value (mer)'] - merged['value']) / merged['value']
merged['%_change_roi'] = 100 * (merged['roi (mer)'] - merged['roi']) / merged['roi']

# Optional: format as string with 2 decimals
merged['spend'] = merged['spend'].map('{:,.2f}'.format)
merged['value'] = merged['value'].map('{:,.2f}'.format)
merged['value (mer)'] = merged['value (mer)'].map('{:,.2f}'.format)
merged['roi'] = merged['roi'].map('{:,.1f}'.format)
merged['roi (est)'] = merged['roi (est)'].map('{:,.1f}'.format)
merged['roi (mer)'] = merged['roi (mer)'].map('{:,.2f}'.format)

merged['%_change_spend'] = merged['%_change_spend'].map('{:+.1f}%'.format)
merged['%_change_value'] = merged['%_change_value'].map('{:+.1f}%'.format)
merged['%_change_roi'] = merged['%_change_roi'].map('{:+.1f}%'.format)

merged

,channel,value,spend,roi,roi (est),% change (est/actual),spend (mer),value (mer),roi (mer),%_change_spend,%_change_value,%_change_roi
0,m_wow_tv,"4,471,876.00","2,699,491.00",1.7,1.3,-24.10,2699491.0,"1,063,363.00",0.40,+0.0%,-76.2%,-75.9%
1,m_wow_olv,"1,923,657.00","947,317.00",2.0,3.8,89.16,947317.0,"797,647.00",0.80,+0.0%,-58.5%,-60.6%
2,m_wow_social,"497,480.00","121,919.00",4.1,3.4,-16.42,121919.0,"186,071.00",1.50,+0.0%,-62.6%,-63.2%
3,m_amaze_tot,"10,117,907.00","7,929,905.00",1.3,1.3,0.78,7929905.0,"5,448,716.00",0.70,+0.0%,-46.1%,-45.3%
4,m_celeb_tv,"8,590,703.00","2,828,021.00",3.0,1.1,-62.83,2828021.0,"50,508,536.00",17.90,+0.0%,+487.9%,+488.8%
5,m_celeb_outdoor,"3,712,733.00","1,290,619.00",2.9,2.2,-22.92,1290619.0,"13,401,968.00",10.40,+0.0%,+261.0%,+261.1%
6,m_celeb_display,"3,941,598.00","1,586,718.00",2.5,0.8,-69.76,1586718.0,"13,889,397.00",8.80,+0.0%,+252.4%,+254.8%
7,Total,"33,255,954.00","17,403,991.00",1.9,1.4,-25.13,17403992.0,"85,295,560.00",4.90,+0.0%,+156.5%,+156.5%


In [31]:
merged_rois = merged[['channel', 'spend', 'roi',  'roi (est)', 'roi (mer)']]

merged_rois

,channel,spend,roi,roi (est),roi (mer)
0,m_wow_tv,"2,699,491.00",1.7,1.3,0.40
1,m_wow_olv,"947,317.00",2.0,3.8,0.80
2,m_wow_social,"121,919.00",4.1,3.4,1.50
3,m_amaze_tot,"7,929,905.00",1.3,1.3,0.70
4,m_celeb_tv,"2,828,021.00",3.0,1.1,17.90
5,m_celeb_outdoor,"1,290,619.00",2.9,2.2,10.40
6,m_celeb_display,"1,586,718.00",2.5,0.8,8.80
7,Total,"17,403,991.00",1.9,1.4,4.90
